In [ ]:
%cd ..

In [ ]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import uuid
import os

**Assuming that the browser object is already setup with:**

    opts = Options()
    opts.set_headless()
    assert opts.headless  
    browser = Firefox(options=opts)
    browser.get('HTTPS_ADDRESS_OF_YOUR_CHOICE')

**EXAMPLE 1:**

Find search form field:

    search_form = browser.find_element_by_id('element_id')

Fill in search form field:

    search_form.send_keys('real python')
    
Submit search form field:

    search_form.submit()
    
Checkout the top result with:

    results = browser.find_elements_by_class_name('result')
    print(results[0].text)

Close browser object before exiting your Python session:

    browser.close()
    quit()

**EXAMPLE 2:**

Click on a specific playbutton:

    browser.find_element_by_class('playbutton').click()
    
**EXAMPLE 3:**

Find elements witha specific class attribute, e.g. "discover-item"

    list_of_items = browser.find_elements_by_class_name('discover-item')

In [ ]:
OUTPUT_DIR = "./assets/test_output"
AOI_FILE_PATH = None # This needs to be a .zip file consisting of .shp, .dbf, .shx and .prj 

In [ ]:
opts = Options()
opts.headless = True
# Operating in headless mode
assert opts.headless  
browser = Firefox(options=opts)

In [ ]:
print("...waiting for page to load.")

browser.get("https://environment.data.gov.uk/DefraDataDownload/?Mode=survey")

wait = WebDriverWait(browser, 300)

print("...page has loaded.")

In [ ]:
AOI_FILE_PATH

In [ ]:
print("...waiting for shapefile to load")

wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#fileid")))

browser.find_element(by=By.CSS_SELECTOR, value="#fileid").send_keys([AOI_FILE_PATH])

wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".grid-item-container")))

wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".grid-item-container")))

E1 = browser.find_element(by=By.CSS_SELECTOR, value=".grid-item-container")

In [ ]:
print("...waiting for available products to load")

while True:  # hack :(
    
    try:
        E1.click()
        
    except ElementNotInteractableException as e:
        break

    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#productSelect")))
    
    products = [
        x.get_attribute("value")
        for x in Select(
            browser.find_element(by=By.CSS_SELECTOR, value="#productSelect")
        ).options
    ]

print(products)

In [ ]:
for product in product_list:
    if product not in products:
        print("product not available")
    else:
        xP = '//*[@id="productSelect"]/option[{}]'.format(
            products.index(product) + 1
        )
        wait.until(EC.presence_of_element_located((By.XPATH, xP)))
        driver.find_element(by=By.XPATH, value=xP).click()

        years = [
            x.get_attribute("value")
            for x in Select(
                driver.find_element(by=By.CSS_SELECTOR, value="#yearSelect")
            ).options
        ]
        if year == "latest":
            xY = ['//*[@id="yearSelect"]/option[1]']
            if verbose:
                print(
                    "downloading data for: {}".format(
                        years[int(xY[0].split("[")[-1][:-1]) - 1]
                    )
                )
        elif not all_years:
            if year not in years:
                print(
                    "no {} data available for {}, available years are {}".format(
                        product, year, ", ".join(years)
                    )
                )
                continue
            #                     raise YearError('Years available are {}'.format(years))
            xY = [
                '//*[@id="yearSelect"]/option[{}]'.format(
                    years.index(str(year)) + 1
                )
            ]
        else:
            most_recent = int(years[0])
            if most_recent < int(year):
                #                     raise YearError('Years available are {}'.format(years))
                print(
                    "no {} data available for {}, available years are {}".format(
                        product, year, ", ".join(years)
                    )
                )
                continue
            available_years = [
                str(y) for y in range(int(year), most_recent + 1) if str(y) in years
            ]
            xY = [
                '//*[@id="yearSelect"]/option[{}]'.format(years.index(y) + 1)
                for y in available_years
            ]

        for xYs in xY:
            current = years[int(xYs.split("[")[-1][:-1]) - 1]
            wait.until(EC.presence_of_element_located((By.XPATH, xYs)))
            driver.find_element(by=By.XPATH, value=xYs).click()
            linki = 1
            while True:
                try:
                    href = driver.find_element(
                        by=By.CSS_SELECTOR,
                        value=".data-ready-container > a:nth-child({})".format(
                            linki
                        ),
                    ).get_attribute("href")
                    file_loc = os.path.join(
                        os.path.split(zipf[0])[0]
                        if not download_dir
                        else download_dir,
                        href.split("/")[-1],
                    )
                    if print_only:
                        print("available:", href)
                    else:
                        if not os.path.isfile(file_loc):
                            download_url(href, file_loc)
                            if verbose:
                                print("saved to:", file_loc)
                    linki += 1
                except NoSuchElementException:
                    if verbose and not print_only:
                        print(linki - 1, "files downloaded for {}".format(current))
                    break
                except Exception as err:
                    print(err)

return driver